In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

import_data = pd.read_csv('train.csv')

def preprocess(import_data):

    drop_label = ['ID','ProcessType', 'TransactionNature', 'Type', 'DeclarerID','IssueDateTime',
                'ImporterID', 'SellerID', 'ExpressID', 'DeclarationOfficeID', 'PaymentType','ExportationCountry',
                'BorderTransportMeans',  'OriginCountry', 'DutyRegime', 'DisplayIndicator']

    pre_data = import_data

    for i in range(len(drop_label)):
        pre_data = pre_data.drop(labels=drop_label[i], axis = 1)

    #preprocess [ClassificationID] label
    classification_id = import_data['ClassificationID'].to_numpy()

    for i in range(classification_id.size):
        id_num = classification_id[i]
        convert = str(id_num)
        if len(convert) == 9:
            convert = '0' + convert
        first_two = convert[:2]
        id_num = int(first_two)
        classification_id[i] = id_num

    pre_data['ClassificationID'] = classification_id

    pre_data = pre_data.astype('float')

    return pre_data


pre_data = preprocess(import_data)
pre_data.head()

,ClassificationID,TaxRate,TotalGrossMassMeasure(KG),AdValoremTaxBaseAmount(Won),Fake
0,94.0,8.0,945.0,720373.5,0.0
1,48.0,0.0,3821.0,85590.4,1.0
2,61.0,13.0,160.0,2496.0,0.0
3,95.0,0.0,10.0,10030.0,0.0
4,32.0,6.5,7265.0,6686706.0,0.0


In [ ]:
pre1 = pre_data['ClassificationID'].to_numpy()
for i in range(pre1.size):
    pre1[i] = pre1[i] * 1000
pre_data['ClassificationID'] = pre1
pre_data.head()

pre2 = pre_data['TaxRate'].to_numpy()
for i in range(pre2.size):
    pre2[i] = pre2[i] * 1000
pre_data['TaxRate'] = pre2
pre_data.head()

pre3 = pre_data['TotalGrossMassMeasure(KG)'].to_numpy()
for i in range(pre3.size):
    pre3[i] = pre3[i] * 1000
pre_data['TotalGrossMassMeasure(KG)'] = pre3
pre_data.head()

pre4 = pre_data['AdValoremTaxBaseAmount(Won)'].to_numpy()
for i in range(pre4.size):
    pre4[i] = pre4[i] * 1000
pre_data['AdValoremTaxBaseAmount(Won)'] = pre4
pre_data.head()

,ClassificationID,TaxRate,TotalGrossMassMeasure(KG),AdValoremTaxBaseAmount(Won),Fake
0,94000.0,8000.0,945000.0,7.203735e+08,0.0
1,48000.0,0.0,3821000.0,8.559040e+07,1.0
2,61000.0,13000.0,160000.0,2.496000e+06,0.0
3,95000.0,0.0,10000.0,1.003000e+07,0.0
4,32000.0,6500.0,7265000.0,6.686706e+09,0.0


In [ ]:
##use all train.csv data to train model
analyze_data = pre_data.astype('float')
data_input = analyze_data.drop(labels = 'Fake', axis = 1).to_numpy()
data_target = analyze_data['Fake'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data_input, data_target, random_state = 42)

kn = KNeighborsClassifier(n_neighbors = 3000, weights = 'distance')
kn.fit(train_input, train_target)
kn.score(test_input, test_target)

0.9108826957719615

In [ ]:
kn = KNeighborsClassifier(n_neighbors = 2000, weights = 'distance')
kn.fit(data_input, data_target)

import_test = pd.read_csv('test.csv')
pre_data2 = preprocess(import_test)
analyze_data = pre_data2.astype('float')

test_data = analyze_data.to_numpy()
result = kn.predict(test_data)

df = pd.DataFrame(result)

df.to_csv('result.csv', index=False)